## Cuaderno de creación de lista negra de usuarios

El código definido a continuación tiene como objetivo analizar las publicaciones de la carpeta [`datos`](/datos/), y crear una lista de usuarios que no deberían ser considerados en el análisis de datos.

Se consideraran para esta lista usuarios que sean autores de: 

* Publicaciones hechas para fomentar el cuidado de la salud mental
    * Si bien se pueden encontrar palabras clave que indiquen tendencias suicidas, no necesariamente reflejan problemas mentales en quien las publica
* Publicaciones repetidas (spam o bots)

Para ejecutar las siguientes instrucciones, se empleará el lenguaje de R.

Será de utilidad en el análisis, además, el paquete `tm`, que permite realizar análisis sobre texto.

In [1]:
# Instalar paquetes necesarios
install.packages(
    c(
        "tm",
        "dplyr"
    ),
    repos = "http://cran.us.r-project.org"
)

also installing the dependencies 'rlang', 'tibble', 'vctrs'




: 

: 

In [1]:
# Se cargan las librerías
library('tm')
library('dplyr')

Warning message:
"package 'tm' was built under R version 4.2.3"
Loading required package: NLP


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
# Se carga el archivo de texto
datos_ahorcarme <- read.csv("./datos/ahorcarme_complete.csv")
datos_colgarme <- read.csv("./datos/colgarme_complete.csv")

In [5]:
head(datos_ahorcarme)

User,Content,Date,URL,Coordinates,Place
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
soiiviic,@frandl_ es una poronga me da ganas de ahorcarme con el cable de la tele,2014-12-30 06:27:27,https://twitter.com/soiiviic/status/549814000750702593,NA,NA
gatoacostadou,"Yo bastante tengo con no ahorcarme con el cable del móvil cuando lo enchufo detrás de la cama, como para tener más de una cuenta en Twitter.",2014-12-29 23:46:38,https://twitter.com/gatoacostadou/status/549713130868465664,NA,NA
Lucasnun_,Menos mal que hoy se juega porque esta fecha provoca que quiera ahorcarme,2014-12-28 17:58:00,https://twitter.com/Lucasnun_/status/549263006409195520,NA,NA
cigarettommy,cada vez falta menos para el cumpleaños de zayn y quiero ahorcarme,2014-12-27 05:56:40,https://twitter.com/cigarettommy/status/548719091817918466,NA,NA
lwtironic,@Z4YNSHADY y hoy me entere que viven juntos queria ahorcarme entiendes lo dificil que es mi vida,2014-12-26 09:40:05,https://twitter.com/lwtironic/status/548412925967609857,NA,NA
CyanideMoon,estoy superfrustrada dadme una sola razón para no ahorcarme,2014-12-25 12:55:53,https://twitter.com/CyanideMoon/status/548099813502623744,NA,NA


In [6]:
# Obtenemos del dataframe las que tengan usuarios que aparezcan más de 5 veces

# Ahorcarme
datos_ahorcarme <- datos_ahorcarme %>%
    group_by(User) %>%
    filter(n() > 5)

# Colgarme
datos_colgarme <- datos_colgarme %>%
    group_by(User) %>%
    filter(n() > 5)

In [4]:
# Creamos la lista de los usuarios que aparecen más de 5 veces

# Ahorcarme
usuarios_ahorcarme <- unique(datos_ahorcarme$User)

# Colgarme
usuarios_colgarme <- unique(datos_colgarme$User)

Ahora, podemos revisar manualmente los tweets de los usuarios que colocamos en la lista negra bajo este criterio, para determinar si realmente sus publicaciones son despreciables para el análisis.

In [4]:
# Obtenemos los tweets de estos usuarios, para revisarlos manualmente

# Ahorcarme
tweets_ahorcarme <- datos_ahorcarme %>%
    filter(User %in% usuarios_ahorcarme)
    # %>%
    # select(User, Content)

# Colgarme
tweets_colgarme <- datos_colgarme %>%
    filter(User %in% usuarios_colgarme)
    # %>%
    # select(User, Content)

ERROR: [1m[33mError[39m in `filter()`:[22m
[1m[22m[36mℹ[39m In argument: `User %in% usuarios_ahorcarme`.
[36mℹ[39m In group 1: `User = "0800merlina"`.
[1mCaused by error in `User %in% usuarios_ahorcarme`:[22m
[33m![39m objeto 'usuarios_ahorcarme' no encontrado


In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
write.table(
    tweets_ahorcarme,
    file = "./datos_limpios/tweets_ahorcarme.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)

# Colgarme
write.table(
    tweets_colgarme,
    file = "./datos_limpios/tweets_colgarme.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)


Luego de revisar los tweets, encontré un fragmento que hace alusión a una connotación sexual de la palabra concurrente. Por lo tanto, aquellos usuarios cuyas publicaciones que contienen la palabra "ahorcarme" serán añadidos a la lista negra.

Es importante considerar que, si bien emplean la palabra bajo un concepto sexual, puede no ser la única forma en la que la usan; de hecho, logré encontrar varios perfiles en los que hacen un uso sexual y violento del verbo. De tal manera, de este grupo de usuarios que tergiversan de esta forma la acción, únicamente se seleccionarán aquellos que emplean la palabra exclusivamente en un contexto sexual, es decir, en todos los tweets que les fueron recolectados que les pertenecen.

In [6]:
# Función que obtiene los tweets de un usuario y determina si todos contienen un texto clave

tweetsSonRepetitivos <- function(usuario, textoClave, dataset) {
    # Obtenemos los tweets del usuario
    tweets <- dataset %>%
        filter(User %in% usuario) %>%
        select(Content)
    
    # Revisamos si todos los tweets contienen el texto clave
    todosContienen <- TRUE
    for (tweet in tweets$Content) {
        if (!grepl(textoClave, tweet)) {
            todosContienen <- FALSE
            break
        }
    }
    
    return (todosContienen)
}

In [7]:
# Fragmento
frag_frase = "ahorcame dios ahorcame hasta el punto de que tus manos"

# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados, siempre y cuando todos los tweets del usuario contengan la frase

# Ahorcarme
usuarios_ahorcarme <- tweets_ahorcarme %>%
    select(User) %>%
    unique()

usuarios_frag_ahorcarme <- c()

In [ ]:
for (usuario in usuarios_ahorcarme$User) {

    #Obtenemos los tweets del usuario
    tweets <- tweets_ahorcarme %>%
        filter(User == usuario) %>%
        select(Content)
    
    # Revisamos si todos los tweets contienen el texto clave
    todosContienen <- TRUE
    for (tweet in tweets$Content) {
        if (!grepl(frag_frase, tweet)) {
            todosContienen <- FALSE
            break
        }
    }
    
    if (todosContienen) {
        usuarios_frag_ahorcarme <- c(usuarios_frag_ahorcarme, usuario)
    }
    
}

Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables: `User`
Adding missing grouping variables:

: 

: 

In [ ]:
# Agregar los usuarios a la lista de usuarios

# Ahorcarme
usuarios_ahorcarme <- c(usuarios_ahorcarme, usuarios_frag_ahorcarme)

# Colgarme
usuarios_colgarme <- c(usuarios_colgarme, usuarios_frag_colgarme)

A partir de aquí, podemos revisar la lista negra de usuarios y verficar uno a uno si estos usuarios deben ser considerados en el análisis.

In [7]:
# Excluir a estos usuarios de los datasets, pero a partir de la lista negra de usuarios ya revisada

# Leer lista negra de usuarios
lista_negra_ahorcarme <- readLines("./datos_limpios/lista_negra_ahorcarme_revisada.txt", encoding = "UTF-8")
# lista_negra_colgarme <- readLines("./datos_limpios/lista_negra_colgarme_revisada.txt", encoding = "UTF-8")

# Ahorcarme
datos_ahorcarme <- datos_ahorcarme %>%
    filter(!User %in% lista_negra_ahorcarme)

# Colgarme
# datos_colgarme <- datos_colgarme %>%
#     filter(!User %in% lista_negra_colgarme)

Warning message in readLines("./datos_limpios/lista_negra_ahorcarme_revisada.txt", :
"incomplete final line found on './datos_limpios/lista_negra_ahorcarme_revisada.txt'"


In [9]:
# Guardamos los tweets en un archivo

# Ahorcarme
write.table(
    datos_ahorcarme,
    file = "./datos_limpios/ahorcarme_filtered.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)